## Creating a program in the IR of the stam language

First, let's import the necessary imports.

In [127]:
from hw.cpu.assembler import asm
from sw.basics import asm_interp
from sw.compiler.backend import CompilerBackend
from sw.compiler.parser import IRParser
from sw.verify.stasm_to_chcs import create_chcs
from z3 import And, Ints, Int, Array, IntSort, ForAll, Implies, Or
import z3
import graphviz
z3.set_param(proof=True)
from sw.verify.stasm_to_chcs import create_chcs

p = IRParser()
cb = CompilerBackend()

Now let's write a function in StamlIR that draws a square. First, we'll add the `horiz` and `vert` functions given to us as examples

In [128]:
prog = '''
/* horiz x0 x1 y  */
horiz(3) =
    (if $2 < $1 then 1 else (if $1 < $2 then 0xffff else 0));
    if $-1 then ignore block $2 $0; ignore wait; .horiz ($2 + $-1) $1 $0
           else 0

/* vert x y0 y1  */
vert(3) =
    (if $1 < $0 then 1 else (if $0 < $1 then 0xffff else 0));
    if $-1 then ignore block $2 $1; ignore wait; .vert $2 ($1 + $-1) $0
           else 0

'''

Now lets add our function and main function

In [129]:
prog +='''
/* square x y len  */
square(3) =
    ignore horiz $0 ($0 + $2) $1;  /* top line */
    ignore vert ($0 + $2) $1 ($1 + $2);  /* right line */
    ignore horiz ($0 + $2) $0 ($1 + $2);  /* bottom line */
    ignore vert $0 ($1 + $2) $1;  /* left line */
    0

main(0) =
    ignore square 5 5 10  /* top line */
'''
prog = '''
main(0) =
    1
'''

Now lets compile and print the code

In [130]:
cb.funcs(p(prog))
cb.code

['main', ('PUSH', 1), ('POP', 2), ('RET', 1)]

Now let's run it on the machine

In [131]:
inter = asm_interp.AsmInterp()
code = cb.code
code =   cb.code
inter.execute_program(code)
print(inter.memory, inter.stack)

RuntimeError: Stack underflow

After we have the code, let's verify that all 4 corners of the rectangle are white

In [112]:
# define a function that draws a square in IR code
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')


chcs = create_chcs(pre_condition=stack[0] == 0,
                   input_vars=[],
                   program=prog,
                   post_condition=And(
                                      memory[0] == 2,
                                      r1 == 2))

ValueError: Invalid program item: 
